**INSTALAMOS LIBRERIAS PARA CARGA**

In [ ]:
! pip install psycopg2
! pip install pgdb 

     |████████████████████████████████| 3.0 MB 4.5 MB/s 
  Created wheel for pgdb: filename=pgdb-0.0.11-py3-none-any.whl size=2800 sha256=a9f5899e6f2f07ce7bdae3842da7a72af0f9ae85dc996ce3caeee33f4aa68289
  Stored in directory: /root/.cache/pip/wheels/ff/83/6a/8cd1726f08b2d24a023f99cdb271b156b9d61ab3771ed80e30
Successfully built pgdb


**ASIGNAMOS EL PROYECTO DONDE SE ENCUENTRA NUESTRA BD Y EL BUCKET DE LOS DATOS QUE SE CARGARAN**

In [ ]:
PROJECT_ID = "keepcoding-proyectofinal-01" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
BUCKET_NAME = "kcp-bucket-01" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://kcp-bucket-01/datos_detection/
                                 gs://kcp-bucket-01/datos_equivalencies/
                                 gs://kcp-bucket-01/datos_images_out/
                                 gs://kcp-bucket-01/datos_shopping_list/
                                 gs://kcp-bucket-01/datos_store/


**COPIAMOS LA CARPETA DATOS DE NUESTRO BUCKET A NUESTRO LOCAL PARA TRATARLOS ANTES DE REALIZAR LA CARGA**

In [ ]:
! gsutil cp -r gs://$BUCKET_NAME/datos_store /content/

Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_04042022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_07042022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_22032022.xlsx...
| [3 files][134.1 KiB/134.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_23032022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_24032022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_28032022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_29032022.xlsx...
Copying gs://kcp-bucket-01/datos_store/supermarket_scrapping_en_30032022.xlsx...
/ [8 files][350.2 K

LEEMOS EL FICHERO CSV DE STORES A UN DATAFRAME

In [ ]:
import os
import pandas as pd
import glob

files_joined = os.path.join('/content/datos_store', '*.xlsx')
list_files_store = glob.glob(files_joined)
df_data_stores = pd.concat(map(pd.read_excel, list_files_store), ignore_index=True)

TRATAMOS LOS DATOS DEL DATAFRAME ANTES DE LA CARGA

In [ ]:
df_data_stores['Peso [gramos]'][df_data_stores['Peso [gramos]'] == 'unidad'] = 1000
df_data_stores['Peso [gramos]'][df_data_stores['Peso [gramos]'] == 0] = 1000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_data_stores = df_data_stores.dropna()

In [ ]:
df_data_stores.reset_index(drop=True, inplace=True)

In [ ]:
df_data_stores['Precio/Kg'] = df_data_stores['Precio'] * 1000 / df_data_stores['Peso [gramos]']
df_data_stores['Precio/Kg'] = pd.Series(["{0:.2f}".format(val) for val in df_data_stores['Precio/Kg']])

COMPROBAMOS LOS DATOS DEL DATAFRAME

In [ ]:
print(df_data_stores.columns)
df_data_stores.head()

Index(['Supermercado', 'Tipo', 'Fecha', 'Categoria', 'Category',
       'SubCategoria', 'Precio', 'Peso [gramos]', 'Precio/Kg'],
      dtype='object')


,Supermercado,Tipo,Fecha,Categoria,Category,SubCategoria,Precio,Peso [gramos],Precio/Kg
0,Carrefour,fruta,2022-03-24,banana,banana,granel,1.25,1000,1.25
1,Carrefour,fruta,2022-03-24,fresón,strawberry,tarrina,2.29,1000,2.29
2,Carrefour,fruta,2022-03-24,aguacate,avocado,carrefour,1.99,1000,1.99
3,Carrefour,fruta,2022-03-24,uva,grape,blanca,2.40,1000,2.40
4,Carrefour,fruta,2022-03-24,manzana,apple,golden,1.95,1000,1.95


GENERAMOS UN NUEVO CSV CON LOS DATOS CON EL NUEVO FORMATO. APROVECHAMOS PARA QUITAR EL INDICE.

In [ ]:
df_data_stores.to_csv('/content/datos_store/Datos_Scrapping_Stores_format.csv', index=False, sep=',')

**IMPORTAMOS LIBRERIAS PARA TRABAJAR CON LA CARGA DE DATOS**

In [ ]:
import psycopg2
import pgdb

**CARGA DE DATOS EN LA TABLA STORES**

In [ ]:
def cargar(path):
    """ Conexión al servidor de pases de datos PostgreSQL """
    conexion = None
    try:
        # Lectura de los parámetros de conexion
        #params = config()
 
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        conexion = pgdb.Connection(host="34.79.94.25",database="proyecto", user="postgres", password="keepcoding")
 
        # creación del cursor
        cur = conexion.cursor()

        # def_table = '''
        # DROP TABLE IF EXISTS stores;  
        # CREATE TABLE IF NOT EXISTS Stores
        # (     supermercado TEXT NOT NULL,
        #       tipo TEXT NOT NULL,
        #       fecha DATE NOT NULL,
        #       categoria TEXT NOT NULL,
        #       category TEXT NOT NULL,
        #       subcategoria TEXT NOT NULL,
        #       precio NUMERIC NOT NULL,
        #       peso_gr NUMERIC NOT NULL,
        #       precio_kg NUMERIC NOT NULL
        # );'''
        # cur.execute(def_table)
        # conexion.commit()

        csv_file_name = path
        sql = "COPY Stores FROM STDIN DELIMITER ',' CSV HEADER"
        cur.copy_expert(sql, open(csv_file_name, "r"))
        conexion.commit()

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            conexion.close()
            print('Conexión finalizada.')

LLAMAMOS A LA FUNCION DE CARGA CON EL FICHERO CORRESPONDIENTE

In [ ]:
cargar('/content/datos_store/Datos_Scrapping_Stores_format.csv')

Conectando a la base de datos PostgreSQL...
Conexión finalizada.


**SE MUEVEN LOS DATOS A LA CARPETA PROCESADOS**

In [ ]:
! gsutil mv -r gs://$BUCKET_NAME/datos_shopping_list gs://$BUCKET_NAME/datos_processed/